In [1]:
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
import os

# own code library
from config.config import *
from config.dataprep_config import *
from plotting import *
from dataprep.preprocessors import *

In [2]:
abspath = r"C:\Users\Andy\PycharmProjects\finrlpaper2\MT-DRL-Pytorch"

# Preprocessing US stock dataset
Using intermediate dataset, where unfit company tics have already been removed.

In [3]:
df_us = pd.read_csv(os.path.join(abspath, "data", "intermediate", "US_stocks_WDB_a.csv"), index_col=0)
df_us.head()

,iid,datadate,tic,conm,ajexdi,cshoc,cshtrd,eps,prccd,prchd,prcld,prcod,prcstd,trfd,log_prccd
gvkey,,,,,,,,,,,,,,,
1690,1,19950103,AAPL,APPLE INC,112.0,NaN,927400.0,NaN,38.375,38.875,37.875,NaN,3.0,1.083307,3.647406
1690,1,19950104,AAPL,APPLE INC,112.0,NaN,1416800.0,NaN,39.375,39.625,38.625,NaN,3.0,1.083307,3.673131
1690,1,19950105,AAPL,APPLE INC,112.0,NaN,657500.0,NaN,38.875,39.375,38.750,NaN,3.0,1.083307,3.660351
1690,1,19950106,AAPL,APPLE INC,112.0,NaN,9613000.0,NaN,42.000,43.125,41.125,NaN,3.0,1.083307,3.737670
1690,1,19950109,AAPL,APPLE INC,112.0,NaN,2447000.0,NaN,41.203,41.875,41.000,NaN,3.0,1.083307,3.718511


#### Calculate: adjcp (adjusted closing price), open, high, low, volume

In [4]:
# function from preprocessors.py
df_us = calculate_price_volume_WhartonData(df=df_us.copy(), 
                                   new_cols_subset=data_settings.NEW_COLS_SUBSET, 
                                   target_subset=None)
df_us.head(3)

,iid,datadate,tic,conm,ajexdi,cshoc,cshtrd,eps,prccd,prchd,prcld,prcod,prcstd,trfd,log_prccd,adjcp,open,high,low,volume
0,1,19950103,AAPL,APPLE INC,112.0,NaN,927400.0,NaN,38.375,38.875,37.875,NaN,3.0,1.083307,3.647406,0.342634,NaN,0.347098,0.338170,927400.0
1,1,19950104,AAPL,APPLE INC,112.0,NaN,1416800.0,NaN,39.375,39.625,38.625,NaN,3.0,1.083307,3.673131,0.351562,NaN,0.353795,0.344866,1416800.0
2,1,19950105,AAPL,APPLE INC,112.0,NaN,657500.0,NaN,38.875,39.375,38.750,NaN,3.0,1.083307,3.660351,0.347098,NaN,0.351562,0.345982,657500.0


#### Calculate: technical indicators


In [5]:
# note: this usually takes up to 5 min on my laptop
df_us = add_technical_indicator_with_StockStats(df=df_us, 
                                                technical_indicators_list=["macd", "rsi_21", "cci_21", "dx_21"])
df_us.head(3)

,iid,datadate,tic,conm,ajexdi,cshoc,cshtrd,eps,prccd,prchd,...,log_prccd,adjcp,open,high,low,volume,macd,rsi_21,cci_21,dx_21
0,1,19950103,AAPL,APPLE INC,112.0,NaN,927400.0,NaN,38.375,38.875,...,3.647406,0.342634,NaN,0.347098,0.338170,927400.0,0.000000,NaN,NaN,NaN
1,1,19950104,AAPL,APPLE INC,112.0,NaN,1416800.0,NaN,39.375,39.625,...,3.673131,0.351562,NaN,0.353795,0.344866,1416800.0,0.000200,100.00000,66.666667,100.0
2,1,19950105,AAPL,APPLE INC,112.0,NaN,657500.0,NaN,38.875,39.375,...,3.660351,0.347098,NaN,0.351562,0.345982,657500.0,0.000117,65.57377,28.571429,100.0


#### Calculate: other features, such as trading volume, volatility, return

In [6]:
df_us = add_other_features(df=df_us,
                       features=["returns_volatility", "return_daily", "log_return_daily"],
                       window_days_vola=7, # window is only relevant for volatility, as returns are always daily here
                       min_periods_vola=7, # min periods to be in window for calculatiom, otherwise NaN calculated
                       price_colum=data_settings.MAIN_PRICE_COLUMN,
                       asset_name_column=data_settings.ASSET_NAME_COLUMN)
df_us.rename(columns={"returns_volatility":"ret_vola_7d"}, inplace=True)

In [7]:
print(df_us["ret_vola_7d"].head(10))
df_us["ret_vola_7d"].isna().sum()/29 # for each of the 29 tickers

0         NaN
1         NaN
2         NaN
3         NaN
4         NaN
5         NaN
6         NaN
7    0.045942
8    0.047910
9    0.047445
Name: ret_vola_7d, dtype: float64


7.0

In [8]:
df_us = add_other_features(df=df_us,
                       features=["returns_volatility"],
                       window_days_vola=21, # 21 trading days in a month, usually
                       min_periods_vola=21,
                       price_colum=data_settings.MAIN_PRICE_COLUMN,
                       asset_name_column=data_settings.ASSET_NAME_COLUMN)
df_us.rename(columns={"returns_volatility":"ret_vola_21d"}, inplace=True)

In [9]:
df_us["ret_vola_21d"].head(100)
df_us["ret_vola_21d"].isna().sum()/29 

21.0

In [10]:
df_us = add_other_features(df=df_us,
                       features=["returns_volatility"],
                       window_days_vola=63, # 63 trading days in a quarter, usually
                       min_periods_vola=63,
                       price_colum=data_settings.MAIN_PRICE_COLUMN,
                       asset_name_column=data_settings.ASSET_NAME_COLUMN)
df_us.rename(columns={"returns_volatility":"ret_vola_63d"}, inplace=True)

In [11]:
df_us.head(100)

,iid,datadate,tic,conm,ajexdi,cshoc,cshtrd,eps,prccd,prchd,...,volume,macd,rsi_21,cci_21,dx_21,ret_vola_7d,return_daily,log_return_daily,ret_vola_21d,ret_vola_63d
0,1,19950103,AAPL,APPLE INC,112.0,NaN,927400.0,NaN,38.375,38.875,...,927400.0,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,19950104,AAPL,APPLE INC,112.0,NaN,1416800.0,NaN,39.375,39.625,...,1416800.0,0.000200,100.000000,66.666667,100.000000,NaN,0.026059,0.025725,NaN,NaN
2,1,19950105,AAPL,APPLE INC,112.0,NaN,657500.0,NaN,38.875,39.375,...,657500.0,0.000117,65.573770,28.571429,100.000000,NaN,-0.012698,-0.012780,NaN,NaN
3,1,19950106,AAPL,APPLE INC,112.0,NaN,9613000.0,NaN,42.000,43.125,...,9613000.0,0.001083,89.437284,133.333333,100.000000,NaN,0.080386,0.077318,NaN,NaN
4,1,19950109,AAPL,APPLE INC,112.0,NaN,2447000.0,NaN,41.203,41.875,...,2447000.0,0.001303,75.434550,65.755057,93.898833,NaN,-0.018976,-0.019159,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,1,19950518,AAPL,APPLE INC,112.0,NaN,3318000.0,NaN,43.375,44.125,...,3318000.0,0.014343,63.981126,121.843410,52.221790,0.026084,-0.014205,-0.014306,0.021654,0.027465
96,1,19950519,AAPL,APPLE INC,112.0,NaN,2880000.0,NaN,42.750,43.750,...,2880000.0,0.013883,61.141003,92.554106,37.625478,0.027391,-0.014409,-0.014514,0.021371,0.027451
97,1,19950522,AAPL,APPLE INC,112.0,NaN,3320000.0,NaN,44.125,44.125,...,3320000.0,0.014344,64.755053,96.623298,37.625478,0.028153,0.032164,0.031657,0.020825,0.027349
98,1,19950523,AAPL,APPLE INC,112.0,NaN,2470000.0,NaN,43.875,44.375,...,2470000.0,0.014364,63.625363,98.678855,40.391891,0.015869,-0.005666,-0.005682,0.020886,0.027353


In [12]:
df_us.sort_values(by=["datadate", "tic"], inplace=True)
df_us["adjcp"].tail()

19974    NaN
39950    NaN
98779    NaN
172023   NaN
33291    NaN
Name: adjcp, dtype: float64

In [13]:
df_us.loc[df_us["datadate"]>=20210611, ["datadate", "tic", "adjcp"]]
### TODO: remove Nan, especially after date 14.6.2021: df[df["datadate"]>=20210614] (last load was on 13.6.2021)

,datadate,tic,adjcp
6657,20210611,AAPL,127.35
13315,20210611,AMGN,242.77
19973,20210611,AXP,164.51
26632,20210611,BA,247.28
33290,20210611,CAT,220.70
39949,20210611,CSCO,54.77
46608,20210611,CVX,107.91
53266,20210611,DIS,177.38
58830,20210611,GS,378.05
65488,20210611,HD,310.77


In [14]:
# removing data that goes beyond the latest fetching date (20210611)
df_us = df_us[df_us["datadate"]<=20210611]

In [15]:
df_us["adjcp"].tail()

165363    397.89
172022     57.33
178681     55.31
185339    140.75
191997     62.17
Name: adjcp, dtype: float64

In [16]:
df_us.columns

Index(['iid', 'datadate', 'tic', 'conm', 'ajexdi', 'cshoc', 'cshtrd', 'eps',
       'prccd', 'prchd', 'prcld', 'prcod', 'prcstd', 'trfd', 'log_prccd',
       'adjcp', 'open', 'high', 'low', 'volume', 'macd', 'rsi_21', 'cci_21',
       'dx_21', 'ret_vola_7d', 'return_daily', 'log_return_daily',
       'ret_vola_21d', 'ret_vola_63d'],
      dtype='object')

In [17]:
relevant_cols = ['datadate', 'tic','eps','adjcp', 'open', 'high', 'low', 'volume', 'macd', 'rsi_21', 'cci_21',
                 'dx_21', 'ret_vola_7d', 'return_daily', 'log_return_daily','ret_vola_21d', 'ret_vola_63d']

In [18]:
df_us = df_us[relevant_cols]
df_us.head()

,datadate,tic,eps,adjcp,open,high,low,volume,macd,rsi_21,cci_21,dx_21,ret_vola_7d,return_daily,log_return_daily,ret_vola_21d,ret_vola_63d
0,19950103,AAPL,NaN,0.342634,NaN,0.347098,0.338170,927400.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6658,19950103,AMGN,NaN,7.281250,NaN,7.390625,7.250000,938700.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
13316,19950103,AXP,NaN,9.750000,NaN,9.833333,9.666667,1321700.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
19975,19950103,BA,NaN,23.375000,NaN,23.562500,23.062500,649600.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
26633,19950103,CAT,NaN,13.750000,NaN,13.781250,13.656250,531100.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [19]:
print(f"dataset length: {len(df_us)}")
df_us[df_us.datadate>=20000101].isna().sum() #eps and open have many missing values, even from 2000 on
# hence we drop them

dataset length: 191990


datadate                0
tic                     0
eps                   307
adjcp                   2
open                33874
high                    2
low                     2
volume                  2
macd                    0
rsi_21                  0
cci_21                  2
dx_21                   0
ret_vola_7d             0
return_daily            0
log_return_daily        4
ret_vola_21d            0
ret_vola_63d            0
dtype: int64

In [20]:
df_us.drop(columns=["eps", "open"], inplace=True)

In [21]:
########TODO
df_us = df_us[df_us.datadate>=20000101]

In [22]:
df_us[df_us.datadate>=20000101].isna().sum()

datadate            0
tic                 0
adjcp               2
high                2
low                 2
volume              2
macd                0
rsi_21              0
cci_21              2
dx_21               0
ret_vola_7d         0
return_daily        0
log_return_daily    4
ret_vola_21d        0
ret_vola_63d        0
dtype: int64

In [23]:
df_us[df_us['adjcp'].isna()] # can look up in another data bank or impute, since only one value per ticker missing
# stock: KO, NKE

,datadate,tic,adjcp,high,low,volume,macd,rsi_21,cci_21,dx_21,ret_vola_7d,return_daily,log_return_daily,ret_vola_21d,ret_vola_63d
100469,20010912,KO,NaN,NaN,NaN,NaN,0.505423,62.198522,NaN,47.669003,0.012710,0.0,NaN,0.013867,0.013617
133762,20010913,NKE,NaN,NaN,NaN,NaN,0.069560,51.964271,NaN,9.947278,0.015348,0.0,NaN,0.014570,0.020281


In [24]:
df_us[df_us["high"].isna()] # can lok up in another data bank or impute, since only one value per ticker missing
# stock: KO, NKE

,datadate,tic,adjcp,high,low,volume,macd,rsi_21,cci_21,dx_21,ret_vola_7d,return_daily,log_return_daily,ret_vola_21d,ret_vola_63d
100469,20010912,KO,NaN,NaN,NaN,NaN,0.505423,62.198522,NaN,47.669003,0.012710,0.0,NaN,0.013867,0.013617
133762,20010913,NKE,NaN,NaN,NaN,NaN,0.069560,51.964271,NaN,9.947278,0.015348,0.0,NaN,0.014570,0.020281


In [25]:
df_us[df_us["low"].isna()] # can lok up in another data bank or impute, since only one value per ticker missing
# stock: KO, NKE

,datadate,tic,adjcp,high,low,volume,macd,rsi_21,cci_21,dx_21,ret_vola_7d,return_daily,log_return_daily,ret_vola_21d,ret_vola_63d
100469,20010912,KO,NaN,NaN,NaN,NaN,0.505423,62.198522,NaN,47.669003,0.012710,0.0,NaN,0.013867,0.013617
133762,20010913,NKE,NaN,NaN,NaN,NaN,0.069560,51.964271,NaN,9.947278,0.015348,0.0,NaN,0.014570,0.020281


In [26]:
df_us[df_us["volume"].isna()] # can lok up in another data bank or impute, since only one value per ticker missing
# stock: KO, NKE

,datadate,tic,adjcp,high,low,volume,macd,rsi_21,cci_21,dx_21,ret_vola_7d,return_daily,log_return_daily,ret_vola_21d,ret_vola_63d
100469,20010912,KO,NaN,NaN,NaN,NaN,0.505423,62.198522,NaN,47.669003,0.012710,0.0,NaN,0.013867,0.013617
133762,20010913,NKE,NaN,NaN,NaN,NaN,0.069560,51.964271,NaN,9.947278,0.015348,0.0,NaN,0.014570,0.020281


In [27]:
df_us[df_us["log_return_daily"].isna()] # can lok up in another data bank or impute, since only one value per ticker missing
# stock: KO, NKE

,datadate,tic,adjcp,high,low,volume,macd,rsi_21,cci_21,dx_21,ret_vola_7d,return_daily,log_return_daily,ret_vola_21d,ret_vola_63d
100469,20010912,KO,NaN,NaN,NaN,NaN,0.505423,62.198522,NaN,47.669003,0.012710,0.000000,NaN,0.013867,0.013617
133762,20010913,NKE,NaN,NaN,NaN,NaN,0.069560,51.964271,NaN,9.947278,0.015348,0.000000,NaN,0.014570,0.020281
100470,20010917,KO,25.10000,25.1000,24.35500,10128300.0,0.520251,62.198522,86.400327,47.669003,0.012444,0.005005,NaN,0.013863,0.013619
133763,20010917,NKE,5.50625,5.6875,5.49625,2371700.0,-0.003167,51.964271,-317.938509,9.947278,0.036960,-0.094553,NaN,0.025059,0.023625


In [28]:
### Let us drop KO and NKE, since they have missing values and we already have a lot of stocks in the portfolio
# anyways so it might be better to reduce the number of stocks
df_us = df_us[~df_us["tic"].isin(["KO", "NKE"])]
df_us.tic.unique()

array(['AAPL', 'AMGN', 'AXP', 'BA', 'CAT', 'CSCO', 'CVX', 'DIS', 'GS',
       'HD', 'HON', 'IBM', 'INTC', 'JNJ', 'JPM', 'MCD', 'MMM', 'MRK',
       'MSFT', 'PFE', 'PG', 'RTX', 'UNH', 'VZ', 'WBA', 'WMT', 'XOM'],
      dtype=object)

In [29]:
df_us[df_us.datadate>=20000101].isna().sum() # no missing values anymore, hence we can save it

datadate            0
tic                 0
adjcp               0
high                0
low                 0
volume              0
macd                0
rsi_21              0
cci_21              0
dx_21               0
ret_vola_7d         0
return_daily        0
log_return_daily    0
ret_vola_21d        0
ret_vola_63d        0
dtype: int64

#### VIX (Volatility index)

In [46]:
# add volatility index
vix = pd.read_csv(os.path.join(abspath, "data", "raw", "VIX.csv"), index_col = 0)
vix

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
1/3/2000,24.360001,26.150000,23.980000,24.209999,24.209999,0
1/4/2000,24.940001,27.180000,24.799999,27.010000,27.010000,0
1/5/2000,27.980000,29.000000,25.850000,26.410000,26.410000,0
1/6/2000,26.680000,26.709999,24.700001,25.730000,25.730000,0
1/7/2000,25.139999,25.170000,21.719999,21.719999,21.719999,0
...,...,...,...,...,...,...
6/4/2021,18.090000,18.420000,16.180000,16.420000,16.420000,0
6/7/2021,17.340000,17.350000,15.780000,16.420000,16.420000,0
6/8/2021,16.580000,17.750000,15.150000,17.070000,17.070000,0


In [47]:
# luckily, there are no missing values
vix.isna().sum()

Open         0
High         0
Low          0
Close        0
Adj Close    0
Volume       0
dtype: int64

In [49]:
# let's plot it
#plt.plot(vix["Adj Close"])
#plt.show()
#plt.savefig("vix_adjclose") # in case it doesn't show, just save and open
#plt.clf() # tell matplotlib we are done with this plot so it doesn't append subsqeuent plots

In [50]:
# let's plot it
#np.log(df.price) - np.log(df.price.shift(1))
#plt.plot(vix["Adj Close"].pct_change(1))
#plt.show()
#plt.savefig("vix_change") # in case it doesn't show, just save and open
#plt.clf() # tell matplotlib we are done with this plot so it doesn't append subsqeuent plots

# we see that the change in VIX is not a good proxy for the VIX value, 
# because there are no distinct peaks during crisis in VIX change, while the VIX value peaks
# but we should still somehow bring this value down a little so it is not so large compared to the other values

In [51]:
d = vix["Adj Close"].rolling(window=21).mean()
st = vix["Adj Close"].rolling(window=21).std()
vix["AdjStdroll21"] = vix["Adj Close"] / st
vix["AdjMeanroll21"] = vix["Adj Close"] / d
vix["Adj100"] = vix["Adj Close"] /100

In [35]:
# Note: after plotting all of the above, it becomes apparent that the best way to bring down the VIX to 
# decimals while not changing the nature of the time series is simply dividing by 100; then
# the vix is varying between 0.1 and 0.8 (latter in crisis times)

In [36]:
#plt.plot(vix["Ad1001"])
#plt.show()
#plt.savefig("vix_adj100") # in case it doesn't show, just save and open
#plt.clf() # tell matplotlib we are done with this plot so it doesn't append subsqeuent plots

In [37]:
# let's plot it
#np.log(df.price) - np.log(df.price.shift(1))
#plt.plot(vix["AdjMeanroll21"])
#plt.show()
#plt.savefig("vix_stdadj") # in case it doesn't show, just save and open
#plt.clf() # tell matplotlib we are done with this plot so it doesn't append subsqeuent plots

In [52]:
vix = pd.DataFrame(vix["Adj100"])
vix

,Adj100
Date,
1/3/2000,0.2421
1/4/2000,0.2701
1/5/2000,0.2641
1/6/2000,0.2573
1/7/2000,0.2172
...,...
6/4/2021,0.1642
6/7/2021,0.1642
6/8/2021,0.1707


In [53]:
# need to convert string dates to datetime format to be compatible with the format we have in 
# the other data set
vix = vix.reset_index()
vix["Date"] = pd.to_datetime(vix["Date"], format='%m/%d/%Y')
vix["Date"] = vix["Date"].dt.strftime('%Y%m%d')
vix["Date"] = vix["Date"].astype(int)
vix["Date"]

0       20000103
1       20000104
2       20000105
3       20000106
4       20000107
          ...   
5389    20210604
5390    20210607
5391    20210608
5392    20210609
5393    20210610
Name: Date, Length: 5394, dtype: object

In [54]:
# now we renamce the columns
vix.columns = ["datadate", "adjDiv100"]
vix.head(3)

,datadate,adjDiv100
0,20000103,0.2421
1,20000104,0.2701
2,20000105,0.2641


In [59]:
df_us.datadate

1263      20000103
7921      20000103
14579     20000103
21238     20000103
27896     20000103
            ...   
165363    20210611
172022    20210611
178681    20210611
185339    20210611
191997    20210611
Name: datadate, Length: 145665, dtype: int64

In [61]:
vix.datadate

0       20000103
1       20000104
2       20000105
3       20000106
4       20000107
          ...   
5389    20210604
5390    20210607
5391    20210608
5392    20210609
5393    20210610
Name: datadate, Length: 5394, dtype: int32

In [42]:
# now we want to merge this data set with our US data set

In [63]:
df_us

,datadate,tic,adjcp,high,low,volume,macd,rsi_21,cci_21,dx_21,ret_vola_7d,return_daily,log_return_daily,ret_vola_21d,ret_vola_63d
1263,20000103,AAPL,0.999442,1.004464,0.907924,4783300.0,0.020676,62.689835,71.109369,32.884996,0.041159,0.088754,0.085034,0.037706,0.039918
7921,20000103,AMGN,62.937500,70.000000,62.875000,22916100.0,4.433505,69.256746,162.560084,51.622364,0.055611,0.047867,0.046756,0.040388,0.033829
14579,20000103,AXP,52.416667,54.833333,52.000000,1887700.0,0.758946,51.210306,-13.428284,7.875716,0.027691,-0.054135,-0.055656,0.021863,0.024413
21238,20000103,BA,40.562500,41.687500,39.812500,2637100.0,0.091544,49.165297,67.240227,4.977704,0.020398,-0.021116,-0.021342,0.021983,0.023994
27896,20000103,CAT,24.312500,24.500000,23.843750,2527500.0,-0.376621,50.137377,167.696352,16.830546,0.019601,0.033201,0.032661,0.023378,0.027229
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
165363,20210611,UNH,397.890000,403.360000,394.780000,3094284.0,-0.085413,48.286149,-171.230261,25.750828,0.006814,-0.008967,-0.009007,0.005814,0.010855
172022,20210611,VZ,57.330000,57.550000,57.010000,12923530.0,-0.130330,50.142868,23.438693,5.985309,0.003599,-0.000174,-0.000174,0.005581,0.007572
178681,20210611,WBA,55.310000,55.820000,54.810000,3936324.0,0.214953,56.055460,96.937514,8.610903,0.016722,0.000000,0.000000,0.015847,0.015266
185339,20210611,WMT,140.750000,140.850000,139.860000,8408446.0,0.120465,51.691318,-36.375217,6.314197,0.005872,0.006220,0.006200,0.007404,0.008832


In [66]:
df_full = pd.concat([df_us.set_index("datadate"), vix.set_index("datadate")], axis=1)
df_full

,tic,adjcp,high,low,volume,macd,rsi_21,cci_21,dx_21,ret_vola_7d,return_daily,log_return_daily,ret_vola_21d,ret_vola_63d,adjDiv100
datadate,,,,,,,,,,,,,,,
20000103,AAPL,0.999442,1.004464,0.907924,4783300.0,0.020676,62.689835,71.109369,32.884996,0.041159,0.088754,0.085034,0.037706,0.039918,0.2421
20000103,AMGN,62.937500,70.000000,62.875000,22916100.0,4.433505,69.256746,162.560084,51.622364,0.055611,0.047867,0.046756,0.040388,0.033829,0.2421
20000103,AXP,52.416667,54.833333,52.000000,1887700.0,0.758946,51.210306,-13.428284,7.875716,0.027691,-0.054135,-0.055656,0.021863,0.024413,0.2421
20000103,BA,40.562500,41.687500,39.812500,2637100.0,0.091544,49.165297,67.240227,4.977704,0.020398,-0.021116,-0.021342,0.021983,0.023994,0.2421
20000103,CAT,24.312500,24.500000,23.843750,2527500.0,-0.376621,50.137377,167.696352,16.830546,0.019601,0.033201,0.032661,0.023378,0.027229,0.2421
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20210611,UNH,397.890000,403.360000,394.780000,3094284.0,-0.085413,48.286149,-171.230261,25.750828,0.006814,-0.008967,-0.009007,0.005814,0.010855,NaN
20210611,VZ,57.330000,57.550000,57.010000,12923530.0,-0.130330,50.142868,23.438693,5.985309,0.003599,-0.000174,-0.000174,0.005581,0.007572,NaN
20210611,WBA,55.310000,55.820000,54.810000,3936324.0,0.214953,56.055460,96.937514,8.610903,0.016722,0.000000,0.000000,0.015847,0.015266,NaN


In [68]:
# note that VIX has missing data on last day of time series:
df_full.isna().sum()

tic                  0
adjcp                0
high                 0
low                  0
volume               0
macd                 0
rsi_21               0
cci_21               0
dx_21                0
ret_vola_7d          0
return_daily         0
log_return_daily     0
ret_vola_21d         0
ret_vola_63d         0
adjDiv100           27
dtype: int64

In [70]:
df_full = df_full[df_full.index<20210611]
df_full.isna().sum() # all nan are removed

tic                 0
adjcp               0
high                0
low                 0
volume              0
macd                0
rsi_21              0
cci_21              0
dx_21               0
ret_vola_7d         0
return_daily        0
log_return_daily    0
ret_vola_21d        0
ret_vola_63d        0
adjDiv100           0
dtype: int64

In [ ]:
#save to csv
#pd.to_csv(os.path.join(abspath, "data", "preprocessed", "US_stocks_WDB_full.csv"))

### DJIA Index (for benchmarking)

In [211]:
dow30 = pd.read_csv(os.path.join(abspath, "data", "raw", "DJI.csv"), index_col=0).reset_index()
dow30 = dow30[["Date", "Close"]]
dow30

,Date,Close
0,2000-01-03,11357.50
1,2000-01-04,10997.90
2,2000-01-05,11122.70
3,2000-01-06,11253.30
4,2000-01-07,11522.60
...,...,...
5409,2021-07-02,34786.35
5410,2021-07-06,34577.37
5411,2021-07-07,34681.79
5412,2021-07-08,34421.93


In [212]:
# check for nan
dow30.isna().sum() # there are no missing values

Date     0
Close    0
dtype: int64

In [213]:
# rename columns
dow30.columns = ["datadate", "dow30Close"]
dow30

,datadate,dow30Close
0,2000-01-03,11357.50
1,2000-01-04,10997.90
2,2000-01-05,11122.70
3,2000-01-06,11253.30
4,2000-01-07,11522.60
...,...,...
5409,2021-07-02,34786.35
5410,2021-07-06,34577.37
5411,2021-07-07,34681.79
5412,2021-07-08,34421.93


In [214]:
# check date type => object. needs to be converted to datetime int
dow30.datadate

0       2000-01-03
1       2000-01-04
2       2000-01-05
3       2000-01-06
4       2000-01-07
           ...    
5409    2021-07-02
5410    2021-07-06
5411    2021-07-07
5412    2021-07-08
5413    2021-07-09
Name: datadate, Length: 5414, dtype: object

In [215]:
dow30.datadate = pd.to_datetime(dow30.datadate, format='%Y-%m-%d')
dow30.datadate = dow30.datadate.dt.strftime('%Y%m%d')
dow30.datadate = dow30.datadate.astype(int)
dow30.datadate

0       20000103
1       20000104
2       20000105
3       20000106
4       20000107
          ...   
5409    20210702
5410    20210706
5411    20210707
5412    20210708
5413    20210709
Name: datadate, Length: 5414, dtype: int32

In [216]:
# check datadaes of us data set
df_us.datadate.unique()

array([20000103, 20000104, 20000105, ..., 20210609, 20210610, 20210611],
      dtype=int64)

In [217]:
# shorten dow30 such that it has same time range as df_us dataset
dow30 = dow30[dow30.datadate<=20210611]
dow30

,datadate,dow30Close
0,20000103,11357.50
1,20000104,10997.90
2,20000105,11122.70
3,20000106,11253.30
4,20000107,11522.60
...,...,...
5390,20210607,34630.24
5391,20210608,34599.82
5392,20210609,34447.14
5393,20210610,34466.24


In [207]:
# concat dow30 index with main data set and save
df_full2 = pd.concat([df_us.set_index("datadate"), dow30.set_index("datadate")], axis=1)
df_full2.to_csv(os.path.join(abspath, "data", "preprocessed", "US_stocks_Wharton_full_DJIA.csv"))
df_full2

,tic,adjcp,high,low,volume,macd,rsi_21,cci_21,dx_21,ret_vola_7d,return_daily,log_return_daily,ret_vola_21d,ret_vola_63d,dow30Close
datadate,,,,,,,,,,,,,,,
20000103,AAPL,0.999442,1.004464,0.907924,4783300.0,0.020676,62.689835,71.109369,32.884996,0.041159,0.088754,0.085034,0.037706,0.039918,11357.5
20000103,AMGN,62.937500,70.000000,62.875000,22916100.0,4.433505,69.256746,162.560084,51.622364,0.055611,0.047867,0.046756,0.040388,0.033829,11357.5
20000103,AXP,52.416667,54.833333,52.000000,1887700.0,0.758946,51.210306,-13.428284,7.875716,0.027691,-0.054135,-0.055656,0.021863,0.024413,11357.5
20000103,BA,40.562500,41.687500,39.812500,2637100.0,0.091544,49.165297,67.240227,4.977704,0.020398,-0.021116,-0.021342,0.021983,0.023994,11357.5
20000103,CAT,24.312500,24.500000,23.843750,2527500.0,-0.376621,50.137377,167.696352,16.830546,0.019601,0.033201,0.032661,0.023378,0.027229,11357.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20210611,UNH,397.890000,403.360000,394.780000,3094284.0,-0.085413,48.286149,-171.230261,25.750828,0.006814,-0.008967,-0.009007,0.005814,0.010855,34479.6
20210611,VZ,57.330000,57.550000,57.010000,12923530.0,-0.130330,50.142868,23.438693,5.985309,0.003599,-0.000174,-0.000174,0.005581,0.007572,34479.6
20210611,WBA,55.310000,55.820000,54.810000,3936324.0,0.214953,56.055460,96.937514,8.610903,0.016722,0.000000,0.000000,0.015847,0.015266,34479.6


In [208]:
df_full2.isna().sum() # no missing values

tic                 0
adjcp               0
high                0
low                 0
volume              0
macd                0
rsi_21              0
cci_21              0
dx_21               0
ret_vola_7d         0
return_daily        0
log_return_daily    0
ret_vola_21d        0
ret_vola_63d        0
dow30Close          0
dtype: int64

### Analysis of preprocessed data

In [113]:
# get tickers
df_full2["tic"].unique()

array(['AAPL', 'AMGN', 'AXP', 'BA', 'CAT', 'CSCO', 'CVX', 'DIS', 'GS',
       'HD', 'HON', 'IBM', 'INTC', 'JNJ', 'JPM', 'MCD', 'MMM', 'MRK',
       'MSFT', 'PFE', 'PG', 'RTX', 'UNH', 'VZ', 'WBA', 'WMT', 'XOM'],
      dtype=object)

In [ ]:
# get starting prices for test set of each stock
# test start: 20151001 (Ymd)
# test end: 20210406 

In [120]:
#df_full2[df_full2.index==20151001] # show to check

In [121]:
# start price test set
#list(df_full2[df_full2.index==20151001]["adjcp"].values) # get list for latex table

In [124]:
#df_full2[df_full2.index==20210406] # show to check

In [125]:
# end price test set
#list(df_full2[df_full2.index==20210406]["adjcp"].values) # get list for latex table

In [176]:
df_full2_ = df_full2[(df_full2.index>=20151001)&(df_full2.index<=20210406)]
df_full2_.reset_index(inplace=True)
df_full2_

,datadate,tic,adjcp,high,low,volume,macd,rsi_21,cci_21,dx_21,ret_vola_7d,return_daily,log_return_daily,ret_vola_21d,ret_vola_63d,dow30Close
0,20151001,AAPL,27.395,27.4050,26.8275,63849470.0,-0.213248,43.213665,-142.482641,46.629579,0.015355,-0.006528,-0.006549,0.018394,0.021585,16272.01
1,20151001,AMGN,140.230,140.6400,136.6000,4669185.0,-5.284907,40.669088,-97.725969,45.474166,0.028175,0.013809,0.013714,0.024069,0.022874,16272.01
2,20151001,AXP,73.890,74.6000,73.2100,4736708.0,-0.765425,43.103747,-104.996290,12.020032,0.012065,-0.003238,-0.003243,0.011729,0.015080,16272.01
3,20151001,BA,130.610,131.7890,128.9801,2850888.0,-1.665138,43.430153,-62.100748,21.575320,0.014855,-0.002596,-0.002600,0.013862,0.015428,16272.01
4,20151001,CAT,64.390,66.1000,63.6100,7932525.0,-3.038283,31.633191,-117.979620,52.201438,0.025298,-0.014841,-0.014952,0.020367,0.017881,16272.01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
37444,20210406,UNH,364.370,369.7900,362.7200,2697863.0,7.370597,57.055247,19.275651,7.728576,0.012503,-0.015695,-0.015820,0.010532,0.013329,33430.24
37445,20210406,VZ,59.110,59.1100,58.6800,12187890.0,0.736307,62.000371,132.417609,41.850301,0.007988,0.004077,0.004068,0.009753,0.012216,33430.24
37446,20210406,WBA,55.700,57.0500,55.5250,4663166.0,1.492350,64.547485,139.529297,45.141083,0.020678,-0.019366,-0.019556,0.020748,0.020207,33430.24
37447,20210406,WMT,140.100,140.9482,138.5200,10328330.0,0.706261,57.499379,197.681762,46.171494,0.011346,0.004805,0.004794,0.011360,0.013792,33430.24


In [177]:
df_full2_["datadate"] = pd.to_datetime(df_full2_["datadate"], format='%Y%m%d')
df_full2_["datadate"]

c:\users\andy\pycharmprojects\finrlpaper2\mt-drl-pytorch\finrlpapervenv\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


0       2015-10-01
1       2015-10-01
2       2015-10-01
3       2015-10-01
4       2015-10-01
           ...    
37444   2021-04-06
37445   2021-04-06
37446   2021-04-06
37447   2021-04-06
37448   2021-04-06
Name: datadate, Length: 37449, dtype: datetime64[ns]

In [178]:
df_full2_.set_index("datadate", inplace=True)
df_full2_

,tic,adjcp,high,low,volume,macd,rsi_21,cci_21,dx_21,ret_vola_7d,return_daily,log_return_daily,ret_vola_21d,ret_vola_63d,dow30Close
datadate,,,,,,,,,,,,,,,
2015-10-01,AAPL,27.395,27.4050,26.8275,63849470.0,-0.213248,43.213665,-142.482641,46.629579,0.015355,-0.006528,-0.006549,0.018394,0.021585,16272.01
2015-10-01,AMGN,140.230,140.6400,136.6000,4669185.0,-5.284907,40.669088,-97.725969,45.474166,0.028175,0.013809,0.013714,0.024069,0.022874,16272.01
2015-10-01,AXP,73.890,74.6000,73.2100,4736708.0,-0.765425,43.103747,-104.996290,12.020032,0.012065,-0.003238,-0.003243,0.011729,0.015080,16272.01
2015-10-01,BA,130.610,131.7890,128.9801,2850888.0,-1.665138,43.430153,-62.100748,21.575320,0.014855,-0.002596,-0.002600,0.013862,0.015428,16272.01
2015-10-01,CAT,64.390,66.1000,63.6100,7932525.0,-3.038283,31.633191,-117.979620,52.201438,0.025298,-0.014841,-0.014952,0.020367,0.017881,16272.01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-04-06,UNH,364.370,369.7900,362.7200,2697863.0,7.370597,57.055247,19.275651,7.728576,0.012503,-0.015695,-0.015820,0.010532,0.013329,33430.24
2021-04-06,VZ,59.110,59.1100,58.6800,12187890.0,0.736307,62.000371,132.417609,41.850301,0.007988,0.004077,0.004068,0.009753,0.012216,33430.24
2021-04-06,WBA,55.700,57.0500,55.5250,4663166.0,1.492350,64.547485,139.529297,45.141083,0.020678,-0.019366,-0.019556,0.020748,0.020207,33430.24


In [179]:
sr_list = []
total_return_list = []
maxdd_list = []
avg_dd_days_list = []
ticlist = []
std_list = []

for tic in df_full2_.tic.unique():
    ticlist.append(tic)
    subdf = df_full2_[df_full2_.tic==tic]["adjcp"]
    perf = subdf.calc_stats()
    
    # now we can access the statistics like this, for example: (ann = annualized)
    sharpe_ratio_daily_ann = perf.daily_sharpe
    sr_list.append(sharpe_ratio_daily_ann)
    
    total_return = perf.total_return # = cumulative return
    total_return_list.append(total_return)
    
    std_daily_return_ann = perf.daily_vol
    std_list.append(std_daily_return_ann)
    
    maxdd = perf.max_drawdown
    maxdd_list.append(maxdd)
    
    avg_dd_days = perf.avg_drawdown_days
    avg_dd_days_list.append(avg_dd_days)

c:\users\andy\pycharmprojects\finrlpaper2\mt-drl-pytorch\finrlpapervenv\lib\site-packages\ffn\core.py:2299: RuntimeWarning: divide by zero encountered in true_divide
  res = np.divide(er.mean(), std)
c:\users\andy\pycharmprojects\finrlpaper2\mt-drl-pytorch\finrlpapervenv\lib\site-packages\ffn\core.py:2299: RuntimeWarning: divide by zero encountered in true_divide
  res = np.divide(er.mean(), std)
c:\users\andy\pycharmprojects\finrlpaper2\mt-drl-pytorch\finrlpapervenv\lib\site-packages\ffn\core.py:2299: RuntimeWarning: divide by zero encountered in true_divide
  res = np.divide(er.mean(), std)


In [158]:
#ticlist # to check if all correct order

In [ ]:
#sr_list
#total_return_list
#list(np.round(np.array(std_list), 2))
#list(np.round(np.array(maxdd_list), 2))
#list(np.round(np.array(avg_dd_days_list), 2))

In [ ]:
### do the same calculations for the index

dow30.datadate = pd.to_datetime(dow30.datadate, format='%Y-%m-%d')
dow30.datadate = dow30.datadate.dt.strftime('%Y%m%d')
dow30.datadate = dow30.datadate.astype(int)
dow30.datadate

In [ ]:
dow30_ = dow30[(dow30.datadate>=20151001)&(dow30.datadate<=20210406)]
dow30_.datadate = pd.to_datetime(dow30_.datadate, format='%Y%m%d')
dow30_.datadate

In [ ]:
perf = dow30_.set_index("datadate")["dow30Close"].calc_stats()

# now we can access the statistics like this, for example: (ann = annualized)
sharpe_ratio_daily_ann = perf.daily_sharpe
total_return = perf.total_return
avg_daily_return_ann = perf.daily_mean
std_daily_return_ann = perf.daily_vol
maxdd = perf.max_drawdown
avg_dd = perf.avg_drawdown
avg_dd_days = perf.avg_drawdown_days

In [ ]:
# START AND END PRICE
# start price test set
dow30[dow30.datadate==20151001] 

In [ ]:
# end price test set
dow30[dow30.datadate==20210406]

In [ ]:
sharpe_ratio_daily_ann

In [ ]:
total_return

In [ ]:
std_daily_return_ann

In [ ]:
maxdd

In [ ]:
avg_dd_days

In [227]:
### do the same calculations for the index

dow30.datadate = pd.to_datetime(dow30.datadate, format='%Y-%m-%d')
dow30.datadate = dow30.datadate.dt.strftime('%Y%m%d')
dow30.datadate = dow30.datadate.astype(int)
dow30.datadate

0       20000103
1       20000104
2       20000105
3       20000106
4       20000107
          ...   
5390    20210607
5391    20210608
5392    20210609
5393    20210610
5394    20210611
Name: datadate, Length: 5395, dtype: int32

In [228]:
dow30_ = dow30[(dow30.datadate>=20151001)&(dow30.datadate<=20210406)]
dow30_.datadate = pd.to_datetime(dow30_.datadate, format='%Y%m%d')
dow30_.datadate

c:\users\andy\pycharmprojects\finrlpaper2\mt-drl-pytorch\finrlpapervenv\lib\site-packages\pandas\core\generic.py:5494: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


3961   2015-10-01
3962   2015-10-02
3963   2015-10-05
3964   2015-10-06
3965   2015-10-07
          ...    
5343   2021-03-30
5344   2021-03-31
5345   2021-04-01
5346   2021-04-05
5347   2021-04-06
Name: datadate, Length: 1387, dtype: datetime64[ns]

In [230]:
perf = dow30_.set_index("datadate")["dow30Close"].calc_stats()

# now we can access the statistics like this, for example: (ann = annualized)
sharpe_ratio_daily_ann = perf.daily_sharpe
total_return = perf.total_return
avg_daily_return_ann = perf.daily_mean
std_daily_return_ann = perf.daily_vol
maxdd = perf.max_drawdown
avg_dd = perf.avg_drawdown
avg_dd_days = perf.avg_drawdown_days

In [237]:
# START AND END PRICE
# start price test set
dow30[dow30.datadate==20151001] 

,datadate,dow30Close
3961,20151001,16272.01


In [238]:
# end price test set
dow30[dow30.datadate==20210406]

,datadate,dow30Close
5347,20210406,33430.24


In [231]:
sharpe_ratio_daily_ann

0.764814320341371

In [232]:
total_return

1.0544628475523306

In [233]:
std_daily_return_ann

0.1966594339524857

In [239]:
maxdd

-0.3708617047844063

In [240]:
avg_dd_days

23.186666666666667